In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import multiprocessing as mp
import pickle 
import warnings 
warnings.filterwarnings('ignore')

from imports import*
from utils import *
from logistic_regression import *
from rnn import *

from hybrid_sim import *
from hybrid_fit import *
from hybrid_predict import *

from habit_sim import *
from habit_fit import *
from habit_predict import *

from kdh_sim import *
from kdh_fit import *
from kdh_predict import *


import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [2]:
# num of agent
num_of_agents = 1

# num of block
num_of_block = 3

# num of trials 
num_of_trials = 200

# for cross valdation 
array = np.arange(num_of_block)
cv = [np.roll(array,i) for i in range(num_of_block)]
cv = np.array(cv)

models = {

    'hybrid':[ configuration_parameters_hybrid,
               hybrid_sim,
               hybrid_fit,
               hybrid_predict],
    
    'habit':[  configuration_parameters_habit,
               habit_sim,
               habit_fit,
               habit_predict],
    
    'kdh':  [  configuration_parameters_kdh,
               kdh_sim,
               kdh_fit,
               kdh_predict]
}


def bce(y_hat,y_true):
    eps = 1e-7
    return -np.sum( y_true*np.log(y_hat+eps) + (1-y_true)*np.log(1-y_hat+eps) )

In [3]:
# sim
for m in models:
    print(f'Model {m}')
    
    data_per_agent = []
    parameters = []

    for agent in tqdm(range(num_of_agents)):
        param = models[m][0]()
        parameters.append(param)
        
        data = []
        for i in range(num_of_block):
            # create rewards probs 
            reward_probs = create_rndwlk(4,num_of_trials)
            df = models[m][1](
                            param,
                            num_of_trials,
                            reward_probs
            ) 
            data.append(df)
        data_per_agent.append(data)

    df = pd.DataFrame(parameters)
    df.to_csv(f'../results/{m}/{m}_parameters.csv')
    
    for agent in range(num_of_agents):
        for block in range(num_of_block):
            data_per_agent[agent][block].to_csv(f'../data/{m}/{m}_agent_{agent}_sim_{block}.csv')
    

Model hybrid


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 15.08it/s]


Model habit


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 31.10it/s]


Model kdh


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 40.62it/s]


In [4]:
# upload data
all_data = [] 
for sim in range(num_of_block):
    data_per_block = []
    for m in tqdm(models): 
        for agent in range(num_of_agents):
            data_per_block.append((pd.read_csv(f'../data/{m}/{m}_agent_{agent}_sim_{sim}.csv')))
    all_data.append(data_per_block)
    
block_0 = all_data[0]
block_1 = all_data[1]
block_2 = all_data[2]

all_blocks = [block_0,block_1,block_2]

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 386.69it/s]


In [13]:
data_results = {
    
   'agent': [],
   'model': [],
   'train_block': [],
    
   'train_nll_hybrid' : [],
   'val_nll_hybrid' : [],
   'test_nll_hybrid': [], 

   'train_nll_habit' : [], 
   'val_nll_habit' : [],
   'test_nll_habit': [], 

   'train_nll_kdh' : [], 
   'val_nll_kdh' : [],
   'test_nll_kdh': [], 
    
}

K = 5
N = num_of_agents*3 # 3 models
data_results['agent'].append(np.tile(np.arange(0,N),len(cv)))
data_results['train_block'].append(np.repeat(cv[:,0],N))
data_results['model'].append(np.tile(np.repeat(['hybrid','habit','kdh'],num_of_agents),num_of_block))


for m in tqdm(models): 

    print(f'*** Fit with {m} ***')
    
    for train, val, test in cv:
        print(f'*** train {train} | val {val} | test {test} ***')
        
        # fit k times 
        fit_res = []
        for _ in range(K):
            pool = mp.Pool(processes=mp.cpu_count())
            fit = pool.map(models[m][2], all_blocks[train])
            pool.close()
            fit_res.append(fit)
            
        # best train/validation nll    
        all_nll_train = np.zeros(shape=(K,N))
        all_nll_val = np.zeros(shape=(K,N))
        all_nll_test = np.zeros(shape=(K,N))
        best_parameters = [] 
        for k in range(K):
            for n in range(N):
                _ , y_hat, _ = models[m][3](all_blocks[train][n], fit_res[k][n].x)
                nLL = bce(1-y_hat, all_blocks[train][n]['action_stage_1'].values)
                all_nll_train[k,n] =  nLL
                
                _ , y_hat, _ = models[m][3](all_blocks[val][n], fit_res[k][n].x)
                nLL = bce(1-y_hat, all_blocks[val][n]['action_stage_1'].values)
                all_nll_val[k,n] = nLL
                
                _ , y_hat, _ = models[m][3](all_blocks[test][n], fit_res[k][n].x)
                nLL = bce(1-y_hat, all_blocks[test][n]['action_stage_1'].values)
                all_nll_test[k,n] = nLL
                
        best_train = all_nll_train.min(axis=0)
        best_val = all_nll_val.min(axis=0)
        indx = np.argmin(all_nll_val,axis=0)
        best_test = np.array([all_nll_test[indx[n],n] for n in range(N)])

        data_results[f'train_nll_{m}'].append(best_train)
        data_results[f'val_nll_{m}'].append(best_val)
        data_results[f'test_nll_{m}'].append(best_test)
        

for k in data_results:
    data_results[k] = np.concatenate(data_results[k])
df_the = pd.DataFrame(data_results)


  0%|                                                     | 0/3 [00:00<?, ?it/s]

*** Fit with hybrid ***
*** train 0 | val 1 | test 2 ***
*** train 2 | val 0 | test 1 ***
*** train 1 | val 2 | test 0 ***


 33%|███████████████                              | 1/3 [00:15<00:30, 15.15s/it]

*** Fit with habit ***
*** train 0 | val 1 | test 2 ***
*** train 2 | val 0 | test 1 ***
*** train 1 | val 2 | test 0 ***


 67%|██████████████████████████████               | 2/3 [00:23<00:11, 11.14s/it]

*** Fit with kdh ***
*** train 0 | val 1 | test 2 ***
*** train 2 | val 0 | test 1 ***
*** train 1 | val 2 | test 0 ***


100%|█████████████████████████████████████████████| 3/3 [00:28<00:00,  9.43s/it]


In [5]:
K = 5
N = num_of_agents*3 # 3 models

data_results_lr = {
        
   'train_nll_lr' : [],
   'val_nll_lr' : [],
   'test_nll_lr': [], 
    
}

for train, val, test in cv:
    print(f'*** train {train} | val {val} | test {test} ***')
    
    all_nll_train = np.zeros(shape=(K,N))
    all_nll_val = np.zeros(shape=(K,N))
    all_nll_test = np.zeros(shape=(K,N))
    
    fit_res = []    
    for k in range(K):
        cur_res = []
        for n in range(N):
            X, y = preprocess_logistic_regression(all_blocks[train][n],lag=k+1)
            clf, inter, coef = fit_logistic_regression(X,y)
            cur_res.append(clf)
        fit_res.append(cur_res)
            
    # best train/validation nll    
    for k in range(K):
        for n in range(N):
            clf = fit_res[k][n]
            
            # train
            X, y = preprocess_logistic_regression(all_blocks[train][n],lag=k+1)
            if clf == None:
                nLL = -np.log(.5)*200
            else:
                y_hat = clf.predict_proba(X)[:,0]
                nLL = bce(1-y_hat, all_blocks[train][n]['action_stage_1'].values)
            
            all_nll_train[k,n] = nLL
            
            # validation
            X, y = preprocess_logistic_regression(all_blocks[val][n],lag=k+1)
            if clf == None:
                nLL = -np.log(.5)*200
            else:
                y_hat = clf.predict_proba(X)[:,0]
                nLL = bce(1-y_hat, all_blocks[val][n]['action_stage_1'].values)
            
            all_nll_val[k,n] = nLL

            X, y = preprocess_logistic_regression(all_blocks[test][n],lag=k+1)            
            if clf == None:
                nLL = -np.log(.5)*200
            else:
                y_hat = clf.predict_proba(X)[:,0]
                nLL = bce(1-y_hat, all_blocks[test][n]['action_stage_1'].values)
                
            all_nll_test[k,n] = nLL
            
    best_train = all_nll_train.min(axis=0)
    best_val = all_nll_val.min(axis=0)
    indx = np.argmin(all_nll_val,axis=0)
    best_test = np.array([all_nll_test[indx[n],n] for n in range(N)])
    
    data_results_lr[f'train_nll_lr'].append(best_train)
    data_results_lr[f'val_nll_lr'].append(best_val)
    data_results_lr[f'test_nll_lr'].append(best_test)
    
for k in data_results_lr:
    data_results_lr[k] = np.concatenate(data_results_lr[k])
df_lr = pd.DataFrame(data_results_lr)
            

*** train 0 | val 1 | test 2 ***
*** train 2 | val 0 | test 1 ***
*** train 1 | val 2 | test 0 ***


In [15]:
df = pd.concat([df_the,df_lr],axis=1)
df.to_csv('../results/the_lr.csv')

In [6]:
N = num_of_agents*3 # 3 models

INPUT_SIZE = 5
OUTPUT_SIZE = 2
LERANING_RATE = 0.001

hidden_size = 5
num_layers = 1
epochs = 1000

loss_train, loss_val, loss_test  = [], [], []
ll_train, ll_val, ll_test = [], [], []

for n in tqdm(range(N)):
    for train, val, test in cv:

        train_data = behavior_dataset(all_blocks[train][n])
        val_data = behavior_dataset(all_blocks[val][n])
        test_data = behavior_dataset(all_blocks[test][n])

        train_loader = DataLoader(train_data,shuffle=False,batch_size=len(train_data))
        val_loader = DataLoader(val_data,shuffle=False,batch_size=len(val_data))
        test_loader = DataLoader(test_data,shuffle=False,batch_size=len(test_data))
        
        rnn = GRU_NN(INPUT_SIZE, hidden_size, num_layers, OUTPUT_SIZE)
        rnn, train_loss, train_ll, val_loss, val_ll, test_loss, test_ll = train_model(
            rnn,
            train_loader,
            val_loader,
            test_loader,
            epochs=epochs,
            lr=LERANING_RATE) 
                                                                                                                                       
        loss_train.append(train_loss)
        loss_val.append(val_loss)
        loss_test.append(test_loss)
        
        ll_train.append(train_ll)
        ll_val.append(val_ll)
        ll_test.append(test_ll)
        
    print('Done agent',n)
    

  0%|                                                     | 0/3 [00:00<?, ?it/s]

*** train 0 | val 1 | test 2 ***
*** train 2 | val 0 | test 1 ***
*** train 1 | val 2 | test 0 ***


 33%|███████████████                              | 1/3 [01:08<02:16, 68.40s/it]

*** train 0 | val 1 | test 2 ***
*** train 2 | val 0 | test 1 ***
*** train 1 | val 2 | test 0 ***


 67%|██████████████████████████████               | 2/3 [02:16<01:08, 68.18s/it]

*** train 0 | val 1 | test 2 ***
*** train 2 | val 0 | test 1 ***
*** train 1 | val 2 | test 0 ***


100%|█████████████████████████████████████████████| 3/3 [03:24<00:00, 68.28s/it]


In [16]:
with open('../results/loss_train.pickle', 'wb') as handle:
    pickle.dump(loss_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results/loss_val.pickle', 'wb') as handle:
    pickle.dump(loss_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results/loss_test.pickle', 'wb') as handle:
    pickle.dump(loss_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results/ll_train.pickle', 'wb') as handle:
    pickle.dump(ll_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../results/ll_val.pickle', 'wb') as handle:
    pickle.dump(ll_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results/ll_test.pickle', 'wb') as handle:
    pickle.dump(ll_test, handle, protocol=pickle.HIGHEST_PROTOCOL)